### IMPORT LIBRARIES AND SET UP

In [1]:
import tweepy
from tweepy import Stream, OAuthHandler, StreamListener
import json
import time
from collections import Counter
import re
import pandas as pd
from langdetect import detect
import pyprind
import deepdish as dd

In [2]:
#import secret codes
from twitter_pwd import access_token, access_token_secret, consumer_key, consumer_secret

In [3]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

### Get random tweets from a given coordinate box

In [415]:
# 
data_list, texts, langs, locs = [], [], [], []

class StdOutListener(StreamListener):
    """ A listener handles tweets are the received from the stream.
    This is a basic listener that just prints received tweets to stdout.
    """
    
    def __init__(self):
        self.data_list = []
        self.texts = []
        self.langs = []
    
    def on_data(self, data):
        jd = json.loads(data)
        self.data_list.append(jd)
        self.texts.append(jd['text'])
        self.langs.append(jd['lang'])
        try:
            print(data)
            saveFile = open('newtweets.csv', 'a')
            saveFile.write(data).encode("utf8")
            saveFile.write('/n').encode("utf8")
            saveFile.close()
            return True
        except BaseException:
            print ('failed ondata')
            time.sleep(5)

    def on_error(self, status):
        print(status)

In [416]:
#coordinates
Lviv = [23.882904,49.763526,24.163055,49.921167]
Kiev = [30.449982,50.408518,30.639496,50.495958]
Yerevan = [44.329834,40.078071,44.681396,40.296287]
Brussel = [4.258575,50.788575,4.489288,50.913424]
Barcelona = [1.835403,41.375778,2.241898,41.586688]

In [10]:
# #Barcelona
# l_Barc = StdOutListener()
# #ASK FOR KEYWORD TO COLLECT DATA
# stream_Barc = Stream(auth, l_Barc)
# stream_Barc.filter(locations=Barcelona)

In [9]:
#Counter(l_Barc.langs)
# for data, lang in zip(l_Barc.data_list, l_Barc.langs):
#     print(data['user']['location'], lang)
for text in l_Barc.texts:
    if type(text) == str:
        print(text)
    else:
        print(text.decode('utf-8'))

In [10]:
# for ee in l_Barc.data_list:
#     print(ee['place']['id'], ee['place']['place_type'])

In [11]:
# #Brussel
# l_Bru = StdOutListener()
# #ASK FOR KEYWORD TO COLLECT DATA
# stream_Bru = Stream(auth, l_Bru)
# stream_Bru.filter(locations=Brussel)

In [12]:
# #Counter(l_Bru.langs)
# for data, lang in zip(l_Bru.data_list, l_Bru.langs):
#     print(data['user']['location'], lang)

In [13]:
# #LVIV
# l_Lv = StdOutListener()
# #ASK FOR KEYWORD TO COLLECT DATA
# stream_Lv = Stream(auth, l_Lv)
# stream_Lv.filter(locations=Lviv)

In [14]:
# #YEREVAN
# l_Yer = StdOutListener()
# #ASK FOR KEYWORD TO COLLECT DATA
# stream_Yer = Stream(auth, l_Yer)
# stream_Yer.filter(locations=Yerevan)

In [15]:
# for data, lang in zip(l_Yer.data_list, l_Yer.langs):
#     print(data['user']['location'], lang)

In [16]:
# for text in l_Yer.texts:
#     if type(text) == str:
#         print(text)
#     else:
#         print(text.decode('utf-8'))

In [17]:
# for data, lang in zip(data_list, langs):
#     print(data['user']['location'], lang)

In [18]:
#KIEV
l_Kiev = StdOutListener()
#ASK FOR KEYWORD TO COLLECT DATA
stream_Kiev = Stream(auth, l_Kiev)
stream_Kiev.filter(locations=Kiev)

In [19]:
#Counter(l_Kiev.langs)
# for data, lang in zip(l_Kiev.data_list, l_Kiev.langs):
#     print(data['user']['location'], lang)
for text in l_Kiev.texts:
    if type(text) == str:
        print(text)
    else:
        print(text.decode('utf-8'))

In [20]:
for data in l_Kiev.data_list:
    print(data['place']['id'], data['place']['place_type'], 
          data['place']['country'], data['user']['location'])

In [125]:
for text in texts:
    if type(text) == str:
        print(text)
    else:
        print(text.decode('utf-8'))


### KEY FUNCTIONS : USERS, FOLLOWERS, TIMELINES, LANGS

In [4]:
import tweepy
import time
import re

def get_account_network(account_name, rel_type='followers', max_num =100, key_words=None):
    """ Given an account by account_name, 
        find all users that are linked to it via a specified relation type rel_type"""
    pbar = pyprind.ProgBar(max_num)
    list_people = []
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    users = tweepy.Cursor(getattr(api, rel_type, 0), screen_name=account_name).items(max_num)
    while True:
        try:
            user = next(users)
            if not key_words:
                list_people.append(user)
            else:
                locs = '|'.join(key_words)
                patt = re.compile(locs)
                found_loc = re.findall(patt, user._json['location'])
                if found_loc:
                    list_people.append(user)
        except tweepy.TweepError as e:
            if 'Read timed out' in str(e):
                print('fallen here')
                print(e)
                time.sleep(5)
            else:
                time.sleep(60*16)
                user = next(users)
        except StopIteration:
            break            
        pbar.update()
    return list_people
        

In [5]:
def get_account_timeline(account_name, max_num_twts=10):
    """ Given an account name,
        it retrieves a maximum number of tweets stored in a list
        Args:
            * account name: string. Screen_name that identifies the twitter account
            * max_num_twts: integer. Maximum number of tweets to be retrieved for each account
        Returns:
            * list_tweets: list of strings including all retrieved tweets"""
    list_tweets=[]
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    timeline = tweepy.Cursor(api.user_timeline, screen_name=account_name).items(max_num_twts)
    i=0
    while True:
        try:
            tw = next(timeline)
            list_tweets.append(tw)
        except tweepy.TweepError as e:
            if '401' in str(e):    
                print(e)
                time.sleep(10)
                break
            else:
                time.sleep(60*15)
                tw = next(timeline)
        except StopIteration:
            break
    return list_tweets

In [6]:
def get_twts_from_list_account_names(list_accounts, max_num_accounts=None, max_num_twts=10):
    """ """
    pbar = pyprind.ProgBar(len(list_accounts))
    texts_tweets = []
    langs_tweets = []
    authors_tweets = []
    if max_num_accounts:
        list_accounts = list_accounts[:max_num_accounts]
    for idx, f in enumerate(list_accounts):
        #print(idx)
        tl = get_account_timeline(f, max_num_twts=max_num_twts)
        texts_tweets.extend([tw.text for tw in tl])
        langs_tweets.extend([tw.lang for tw in tl])
        authors_tweets.extend([f for _ in tl])
        pbar.update()
    return texts_tweets, langs_tweets, authors_tweets

### UKRAINE: data structure and relevant twitter accounts 

In [12]:
Ukraine_nodes = {}
Ukraine_nodes['cities'] = ['kiev', 'odessa', 'lviv', 'kharkov', 'dnipropetrovsk']
Ukraine_nodes['city_sites'] = {'Mariupol':['0629ComUa'], 
                               'kiev':['kievtypical','kliniki_kiev','LISOD_clinic','avto_kiev', 'editbeauty']}
Ukraine_nodes['news'] = ['HromadskeUA','tsnua','ukrpravda_news', 'lb_ua', 'Korrespondent', 
                         'Delo_ua', 'BBC_ua', 'LIGAnet', 'segodnya_life']
Ukraine_nodes['TV'] = ['5channel', 'EspresoTV', '24tvua', 'footballua_tv']
Ukraine_nodes['starsystem'] = ['VeraBrezhneva', 's_vakarchuk', 'KAMEHCKUX']
Ukraine_nodes['politics'] = ['poroshenko', 'Vitaliy_Klychko', 'andriy_sadovyi', 'GennadyKernes']

In [14]:
key_words=['Україна', 'Ukraine', 'Украина', 'Київ', 'Киев']
HromadskeUA_followers = get_account_network('HromadskeUA', rel_type='followers', 
                                            max_num =5000, key_words=key_words)

In [7]:
country = 'ukr'
node_name = 'politics'
acc_name = 'poroshenko'
rel_type = 'followers'

#key_words=['Україна', 'Ukraine', 'Украина', 'Київ', 'Киев']
path_save = '/'.join(['',country, node_name, acc_name, rel_type])
followers = get_account_network(acc_name, rel_type=rel_type, 
                                max_num =5000, key_words=None)
json_format = [elem._json for elem in followers]
df = pd.DataFrame(json_format)
df.to_hdf('lang_data.h5', path_save)

0%                          100%
[#                             ] | ETA: 00:01:50

Rate limit reached. Sleeping for: 895
fallen here

[###                           ] | ETA: 02:20:33


Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=60)
Rate limit reached. Sleeping for: 891
fallen here

[#####                         ] | ETA: 02:35:13

...


Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=60)
Rate limit reached. Sleeping for: 893
fallen here

[############################  ] | ETA: 00:16:50


Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=60)
Rate limit reached. Sleeping for: 892
fallen here

[##############################] | ETA: 00:00:00
Total time elapsed: 04:12:04

Total time elapsed: 04:12:04

Total time elapsed: 04:12:04

Total time elapsed: 04:12:04

Total time elapsed: 04:12:04

Total time elapsed: 04:12:04

Total time elapsed: 04:12:04

Total time elapsed: 04:12:04

Total time elapsed: 04:12:04

Total time elapsed: 04:12:04

Total time elapsed: 04:12:04

Total time elapsed: 04:12:04

Total time elapsed: 04:12:04

Total time elapsed: 04:12:04

Total time elapsed: 04:12:04

Total time elapsed: 04:12:04

Total time elapsed: 04:12:04
//anaconda/lib/python3.5/site-packages/pandas/core/generic.py:939: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->['created_at', 'description', 'entities', 'id_str', 'lang', 'location', 'name', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_banner_url', 


Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=60)


In [8]:
df[['location','screen_name','lang']][df['location'].str.contains(r"(Львів|Львов)")]

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


,location,screen_name,lang
522,Україна Львів,juliaskab13,uk
1006,Львов,novinska_cat,ru
1661,Львов,HumenuykRoma,ru
2269,Львів,kaprikorn87,uk
2997,Україна Львів,Hollywell83,uk
3803,м .Львів,M8tgs4YGZ391fNw,uk
4731,Львів,cherniak_gi,uk


In [9]:
df['lang'][df['location'].str.contains(r"(Львів|Львов|Lviv|Івано-Франківськ)")].value_counts()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


uk    8
ru    3
en    1
Name: lang, dtype: int64

In [10]:
df['lang'][df['statuses_count'] >= 10][df['location'].str.contains(r"(Kiev|Kyiv|Київ|Киев)")].value_counts()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


ru    14
en     2
uk     1
Name: lang, dtype: int64

In [120]:
df[['lang','screen_name', 'followers_count']][df['statuses_count'] >= 200][df['location'].str.contains(r"(Kiev|Kyiv|Київ|Киев)")]

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':
//anaconda/lib/python3.5/site-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


,lang,screen_name,followers_count
55,ru,AnPrikhodko,209429
139,ru,YouScan,2809
255,ru,Starckad,1906
335,ru,KatiaNeznayko,144
409,ru,shampynelas,34
720,uk,AlexKhrebet,934
756,uk,interfaxua,88046
1204,uk,Yavorskyi_AV,6
1278,ru,vzhernovoy,107
1308,en,edele4ka,264


In [197]:

df[df['statuses_count'] >= 5][['screen_name', 'statuses_count','followers_count']].count()

screen_name        501
statuses_count     501
followers_count    501
dtype: int64

In [78]:
#HRMUA_flwrs = [f for idx, f in df_HRMUA.iterrows()]
#HRMUA_texts, HRMUA_langs = get_twts_from_list_account_names(HRMUA_flwrs, max_num_followers=300)

In [66]:
df_txts_langs_HRMUA = pd.DataFrame({'texts':HRMUA_texts, 'lang':HRMUA_langs})

df_txts_langs_HRMUA.to_hdf('lang_data.h5', '/ukr_nodes/news/HromadskeUA/tls_followers')

In [ ]:
df_txts = pd.read_hdf('lang_data.h5', '/ukr_nodes/news/HromadskeUA/tls_followers')

### CATALONIA NODES

In [56]:
Catalonia_nodes = {}
Catalonia_nodes['news'] = ['LaVanguardia', 'VilaWeb', 'diariARA', 'elperiodico',
                           'elperiodico_cat', 'elpuntavui']
Catalonia_nodes['cities'] = ['bcn_ajuntament', 'paerialleida', 'girona_cat', 'TGNAjuntament', 
                             'AjBadalona', 'aj_vic', 'ajterrassa']

In [87]:
def get_account_followers(country, node, acc_name):
    path_save = '/'.join(['', country, node, acc_name, 'followers'])
    followers = get_account_network(acc_name, rel_type='followers', max_num =5000)
    json_format = [elem._json for elem in followers]
    df = pd.DataFrame(json_format)
    df.to_hdf('lang_data.h5', path_save)
    return df
    

In [88]:
df = get_account_followers(country ='cat', node='cities', acc_name='AjBadalona')

0%                          100%
[#                             ] | ETA: 00:03:21

Rate limit reached. Sleeping for: 889
fallen here

[###                           ] | ETA: 02:20:09


Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=60)
Rate limit reached. Sleeping for: 889
fallen here

[#####                         ] | ETA: 02:38:29

...


Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=60)
Rate limit reached. Sleeping for: 887
fallen here

[############################  ] | ETA: 00:26:34


Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=60)
Rate limit reached. Sleeping for: 887
fallen here

[##############################] | ETA: 00:00:00
Total time elapsed: 06:28:31

Total time elapsed: 06:28:31

Total time elapsed: 06:28:31

Total time elapsed: 06:28:31

Total time elapsed: 06:28:31

Total time elapsed: 06:28:31

Total time elapsed: 06:28:31

Total time elapsed: 06:28:31

Total time elapsed: 06:28:31

Total time elapsed: 06:28:31

Total time elapsed: 06:28:31

Total time elapsed: 06:28:31

Total time elapsed: 06:28:31

Total time elapsed: 06:28:31

Total time elapsed: 06:28:31

Total time elapsed: 06:28:31

Total time elapsed: 06:28:31
//anaconda/lib/python3.5/site-packages/pandas/core/generic.py:939: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->['created_at', 'description', 'entities', 'id_str', 'lang', 'location', 'name', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_banner_url', 


Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=60)


In [89]:
df['lang'].value_counts()

es       3586
ca       1102
en        271
fr          9
it          7
en-gb       5
pt          4
de          3
eu          2
ar          2
ru          2
gl          1
pl          1
en-GB       1
zh-cn       1
ja          1
tr          1
ro          1
Name: lang, dtype: int64

In [90]:
regex = r"(Badal)"
df_Bad = df[df['statuses_count'] >= 10][df['location'].str.contains(regex)]
df_Bad['lang'].value_counts()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  from ipykernel import kernelapp as app
//anaconda/lib/python3.5/site-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


es    688
ca    281
en     51
fr      1
it      1
Name: lang, dtype: int64

In [91]:
281/(688+281)

0.2899896800825593

### GET TWEETS FROM FOLLOWERS

In [95]:
def get_tl_followers(screen_name, country, node_name, city=None, min_num_twts_per_acc=5, max_num_followers=None):
    """ Creates pandas dataframe with all tweet texts and corresponding language
    from followers of a given account. A dataframe with all followers info 
    must have been previously computed and saved in hdf5 format"""
    base_path = '/'.join(['',country, node_name, screen_name])
    path_load = base_path + '/followers'
    path_save = base_path + '/tls_followers'
    key_words = {'ukr':{'all':r"(Україна|Ukraine|Украина|Київ|Киев|Львів|Львов|Одес)", 
                        'Kiev':r"(Kiev|Kyiv|Київ|Киев)"}, 
                 'cat':{'all':r"(Barcel|Catal|Tarr|Llei|Ger|Gir|Badal)",
                        'Terrassa':r"(Terras|Vall)",
                        'Girona':r"(Giro|Gero)",
                        'Vic':r"(Vic|Oson)", 'Barcelona':r"(Barcel|barcel|Bcn|bcn)",
                        'Tarragona':r"(Tarrag|tarrag)", 'Lleida':r"(Lleida|Lerid|Lérid)",
                        'Badalona':r"Badal"}}
    df = pd.read_hdf('lang_data.h5', path_load)
    # filter by num_min_twts_per_account
    relevant_followers = df['screen_name'][df['statuses_count'] >= min_num_twts_per_acc]
    # keep only country residents
    if city:
        relevant_followers = relevant_followers[df['location'].str.contains(key_words[country][city])].values
    else:
        relevant_followers = relevant_followers[df['location'].str.contains(key_words[country]['all'])].values
        
    texts, langs, auth = get_twts_from_list_account_names(relevant_followers, 
                                                          max_num_accounts=max_num_followers)
    df_txts_langs= pd.DataFrame({'texts':texts, 'lang':langs, 'screen_name':auth})
    df_txts_langs.to_hdf('lang_data.h5', path_save)
    return df_txts_langs
    
    

In [96]:
df_txts_langs = get_tl_followers('AjBadalona' , 'cat', 'cities', city='Badalona')

0%                          100%
[##                            ] | ETA: 00:13:58

Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401

[####                          ] | ETA: 00:15:44


Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401
Twitter error response: status code = 401

[#####                         ] | ETA: 00:17:57

...


Twitter error response: status code = 401
Twitter error response: status code = 401

[############################# ] | ETA: 00:00:42


Twitter error response: status code = 401
Twitter error response: status code = 401

[##############################] | ETA: 00:00:00


Total time elapsed: 00:21:29


In [97]:
df_txts_langs['lang'].value_counts()

es     4702
und    4427
en      554
pt       94
fr       40
it       20
in       15
eu       10
nl       10
ht        8
et        6
tl        6
ro        5
hu        5
lt        5
tr        4
cs        4
de        3
cy        3
no        3
da        3
pl        2
sv        2
fi        2
vi        2
hi        1
is        1
ja        1
Name: lang, dtype: int64

In [72]:
from langdetect import detect_langs

In [160]:
import langdetect

In [159]:
detect("jo no vull això")

'fi'

In [171]:
df_txts_langs.keys()

Index(['lang', 'screen_name', 'texts'], dtype='object')

In [98]:
df_txts_langs[['texts', 'lang', 'screen_name']][df_txts_langs['lang'] == 'und']

,texts,lang,screen_name
10,Amb un referèndum el reconeixeria la societat ...,und,davramey
11,Si el Govern d #Badalona està en caiguda lliur...,und,davramey
12,RT @PuigMiquel: @miriamnoguerasM @JordiHD883 @...,und,davramey
13,RT @PereMtzCarrete: El @PDeCATBDN agraïm al pr...,und,davramey
15,RT @nereida_edo: Espectacle patètic d'insults ...,und,davramey
16,"RT @lciuro: En ""47, clar i català, us dic els ...",und,davramey
17,RT @KRLS: Nova vergonya internacional. Es van ...,und,davramey
18,"RT @martapascal: Estrasburg ens ampara, al Par...",und,davramey
19,RT @PDCComarquesBCN: Ds dl @PDCComarquesBCN fe...,und,davramey
20,@JCallauBartoli @AjSantAdria @HugoFerrerSaez @...,und,ferranaracil


In [272]:
base_path = '/'.join(['',country, node_name, screen_name])
path_load = base_path + '/followers'
path_save = base_path + '/tls_followers'

In [280]:
df = pd.read_hdf('lang_data.h5', path_save)

In [230]:
# filter followers to focus on most relevant ones
min_num_twts = 5
relevant_followers = df['screen_name'][df['statuses_count'] >= min_num_twts][
                         df['location'].str.contains(key_words[country])
                       ].values

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [220]:
# get twts, lang, authors. Transform to pandas df and save
texts, langs, auth = get_twts_from_list_account_names(relevant_followers, max_num_followers=79)
df_txts_langs= pd.DataFrame({'texts':texts, 'lang':langs, 'screen_name':auth})
df_txts_langs.to_hdf('lang_data.h5', path_save)

0%                          100%
[#####                         ] | ETA: 00:01:51

Twitter error response: status code = 401
Twitter error response: status code = 401

[#########                     ] | ETA: 00:01:37


Twitter error response: status code = 401

[#############                 ] | ETA: 00:01:22

...


Twitter error response: status code = 401

[########################      ] | ETA: 00:00:33


Twitter error response: status code = 401

[##############################] | ETA: 00:00:00


Total time elapsed: 00:02:40


In [122]:
#df_txts_langs['lang'].value_counts()
#langs_detected = [detect(txt) for txt in df_txts_langs['texts']]

langs_detected=[]
for txt in df_txts_langs['texts']:
    try:
        langs_detected.append(detect(txt))
    except:
        langs_detected.append(None)

In [55]:
df_try = pd.DataFrame({'a':['aaaa','bbbfdde',1],'b':[23,44,56]})
df_try2 = pd.DataFrame({'a':['xxxx','zzzz'],'b':[3233,43214]})

store = pd.HDFStore('try_hyerar.h5')

store.append('city/topic', df_try)

store.close()

pd.read_hdf('try_hyerar.h5', 'city/topic')

store = pd.HDFStore('try_hyerar.h5','a')

store.append('city/topic', df_try2)

store.close()

pd.read_hdf('try_hyerar.h5', 'city/topic')

store = pd.HDFStore('try_hyerar.h5','a')

store.put('city/followers',df)

store.close()

### EXPLORE AN ACCOUNT

In [11]:
api = tweepy.API(auth)
user_info = api.get_user('onlyforulonely ')  #ArnauAndreu
user_info._json['location']

'Ostroh'

In [20]:
#get timeline
tl = get_account_timeline('onlyforulonely',max_num_twts=200)

In [21]:
[(elem._json['lang'],elem._json['text']) for elem in tl]

[('uk', '@anastasiiareshe Подякувала😇'),
 ('uk',
  '@1705Belka тоді чекатиму твоїх смс зі словами "мене не забрав татко, йдемоо"😅'),
 ('uk',
  '@1705Belka тоді зможемо разом піти на випускний 11-их класів, а то у мене немає компанії)\nі платтячка подивимось, і поговоримо)'),
 ('uk', '@1705Belka у Сб будеш ще тут?)'),
 ('uk',
  '@1705Belka в сееенсі одна?\n\nскажи мені, коли ти вільна, і ми зустрінемось😊'),
 ('uk',
  '@1705Belka Лю, я тебе бачила в місті о сьомій, кричала тобі "Люю, ало", а ти говорила по телефону, зірко😦'),
 ('uk', '@sergeykovaliov як варіант😅'),
 ('uk', 'Прикро, коли людина кидається словами на вітер.'),
 ('uk', '@nastya_dubyna дякую, зірко😊😙'),
 ('ru', '@shylodzobkaa Ооой, дякуююю😍'),
 ('uk', '@vika_kopytko Пасііібочкі, кіцьося😍'),
 ('uk',
  '@no_false @Gvrtsss Я тебе запевняю, абсолютно рівноцінний, ти навіть залишаєшся у плюсі😂'),
 ('uk',
  'Дівчата, зацініть😊\rТам сірий, а де білий - мармур😉 https://t.co/bYyQ5GzIj3'),
 ('ru',
  '@no_false @Gvrtsss Ну я тебе зафолл

In [24]:
detect("Ну нічого-нічого, скоро і моя інста стане популярною і всі будуть такі мол огоо Ладка яка ти класна, а я така оо дякую цьом-цьом.")

'uk'

In [12]:
user_info._json

{'contributors_enabled': False,
 'created_at': 'Sun Feb 15 18:20:38 +0000 2015',
 'default_profile': False,
 'default_profile_image': False,
 'description': 'занадто хороша(загадкова) для цього світу. чому б вам не мислити позитивно? бути бажаною, але не бажати нікого. малюю, яжхудожник. make art, be art☀️ #УкрТві',
 'entities': {'description': {'urls': []},
  'url': {'urls': [{'display_url': 'instagram.com/lavoytko/',
     'expanded_url': 'http://www.instagram.com/lavoytko/',
     'indices': [0, 23],
     'url': 'https://t.co/aQVTV4rjd4'}]}},
 'favourites_count': 8970,
 'follow_request_sent': False,
 'followers_count': 742,
 'following': False,
 'friends_count': 407,
 'geo_enabled': True,
 'has_extended_profile': True,
 'id': 3039405867,
 'id_str': '3039405867',
 'is_translation_enabled': False,
 'is_translator': False,
 'lang': 'uk',
 'listed_count': 7,
 'location': 'Ostroh',
 'name': 'Ладка-загадка',
 'notifications': False,
 'profile_background_color': 'FFE1FF',
 'profile_backgroun

In [324]:
my_friends = get_account_network('ArnauAndreu')

my_df = pd.DataFrame(my_followers)

Counter([friend.lang for friend in my_friends])

my_fr_txts, my_friends_lang = get_twts_from_list_account_names(my_friends)

#Counter(my_friends_lang)



# my_fr_langs_detected=[]
# for i,txt in enumerate(my_fr_txts):
#     #print(i, txt)
#     try:
#         my_fr_langs_detected.append(detect(txt))
#     except:
#         continue

#Counter(my_fr_langs_detected)

usr_tl = get_account_timeline(my_friends[44].screen_name, max_num=10)

df_try=pd.DataFrame([twt._json for twt in usr_tl])
df_try.columns

my_tl = get_account_timeline('ArnauAndreu', max_num=10)

### DYNAMO KIEV PLAYERS

In [27]:
# DYNAMO PLAYERS

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

url = "http://www.worldfootball.net/teams/dinamo-kiev/2017/2/"
html = requests.get(url)
soup = BeautifulSoup(html.text, "lxml")

dynamo_players = pd.read_html(url, encoding='utf8')[1][2].dropna().values

dynamo_players

array(['Maksim Koval', 'Artur Rudko', 'Antunes', 'Mykyta Burda',
       'Tamás Kádár', 'Evgen Khacheridi', 'Pavlo Lukyanchuk',
       'Bogdan Mykhaylychenko', 'Zurab Ochihava', 'Aleksandar Pantić',
       'Oleksandr Tymchyk', 'Domagoj Vida', 'Vitaliy Buyalskiy',
       'Valeriy Fedorchuk', 'Denys Garmash', 'Nikita Korzun',
       'Mykyta Kravchenko', 'Mikola Morozyuk', 'Pavel Orekhovskiy',
       'Serhiy Rybalka', 'Volodymyr Shepeliev', 'Sergiy Sydorchuk',
       'Viktor Tsygankov', 'Artem Besedin', 'Derlis González',
       'Roman Yaremchuk', 'Andriy Yarmolenko', 'Sergiy Rebrov'], dtype=object)

In [56]:
dyn_play_countr = pd.read_html(url, encoding='utf8')[1][[2,4]].dropna().values

dyn_play_countr

### PYMONGO

In [5]:
from pymongo import MongoClient

conn=MongoClient()

#define database
db = conn.citylangs

#define collection inside database
collection = db.bcn.ajuntam.followers

# function to add documents to collection 
def make_followers_collection(account_name, collection, max_num=100):
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    users = tweepy.Cursor(api.followers, screen_name=account_name).items(max_num)
    #i = 0
    while True:
        try:
            user = next(users)
            collection.insert_one(user._json)
        except tweepy.TweepError as e:
            if 'Read timed out' in str(e):
                print('fall here')
                print(e)
                time.sleep(5)
            else:
                time.sleep(60*16)
                user = next(users)
        except StopIteration:
            break
        #print ("@" + user.screen_name)
    #return collection

make_followers_collection('bcn_ajuntament',collection, max_num=100)

#check what databases are available
conn.database_names()

# available collections inside db
db.collection_names()

collection2 = db.kiev.ukrpravda

make_followers_collection('ukrpravda_news',collection2, max_num=300)

l = list(conn.citylangs.kiev.ukrpravda.find())

Counter([obj['lang'] for obj in l])


db.collection_names()

#db.categories.insert_one({ "_id": "ukr_pravda", "children": [] })
# db.categories.insert({ _id: "avto_kiev", children: [] })
# db.categories.insert({ _id: "kiev", children: ["ukr_pravda", "avto_kiev"] })

db['kiev'].insert_one({'avto_kiev':[],'vitklitschko':[]})

db.collection_names()

rr=list(db['bcn.ajuntam.followers'].find())

db['kiev'].find_one()

db['bcn.ajuntam.followers']

conn.database_names()

coll2 = db.countries

### MERGE DFs

In [218]:
#get hdf database keys
with pd.HDFStore('lang_data.h5','r') as f:
    my_keys = f.keys()

In [221]:
my_keys

['/ukr_nodes/news/BBC_ua/followers',
 '/ukr_nodes/news/BBC_ua/tls_followers',
 '/ukr_nodes/news/HromadskeUA/followers',
 '/ukr_nodes/news/HromadskeUA/tls_followers',
 '/ukr_nodes/news/LIGAnet/followers',
 '/ukr_nodes/news/LIGAnet/tls_followers',
 '/ukr_nodes/news/ukrpravda_news/followers',
 '/ukr/starsystem/s_vakarchuk/followers',
 '/ukr/starsystem/s_vakarchuk/tls_followers',
 '/ukr/politics/Vitaliy_Klychko/followers',
 '/ukr/politics/Vitaliy_Klychko/tls_followers',
 '/cat_nodes/news/LaVanguardia/followers',
 '/cat_nodes/news/LaVanguardia/tls_followers',
 '/cat_nodes/news/diariARA/followers',
 '/cat_nodes/news/diariARA/tls_followers',
 '/cat/news/elperiodico/followers',
 '/cat/news/elperiodico/tls_followers',
 '/cat/cities/TGNAjuntament/followers',
 '/cat/cities/TGNAjuntament/tls_followers',
 '/cat/cities/aj_vic/followers',
 '/cat/cities/aj_vic/tls_followers',
 '/cat/cities/ajterrassa/followers',
 '/cat/cities/ajterrassa/tls_followers',
 '/cat/cities/bcn_ajuntament/followers',
 '/cat/c

In [234]:
aa=pd.read_hdf('lang_data.h5', '/ukr_nodes/news/BBC_ua/followers')

In [235]:
aa['lang'].value_counts()

ru       2626
uk       1770
en        505
pl         30
ro         13
en-gb      11
hu          6
ar          6
tr          6
es          4
fr          3
ja          3
de          3
pt          3
sk          3
ko          2
bg          1
it          1
nl          1
zh-cn       1
el          1
vi          1
Name: lang, dtype: int64

In [164]:
file_path = 'lang_data.h5'
country = 'cat'
acc_names = ['diariARA', 'LaVanguardia']
load_node1 = '/' + country + '_nodes/news/' + acc_names[0] + '/tls_followers'
load_node2 = '/' + country + '_nodes/news/' + acc_names[1] + '/tls_followers'

df1 = pd.read_hdf(file_path, load_node1)
df2 = pd.read_hdf(file_path, load_node2)

In [165]:
df_merged = pd.merge(df1, df2, how='outer')

In [166]:
df_merged.shape, df1.shape, df2.shape

((44937, 2), (34461, 2), (12181, 2))

In [180]:
%matplotlib
from matplotlib import pyplot as plt


Using matplotlib backend: MacOSX


### TWITTER RANDOM WALK

In [ ]:
#1. Start from mayor, city hall account, etc...

#2. Get a follower from the city (or county, oblast) as first node

#3. Get follower of step2 node

#4. Repeat step3 with new nodes until max number of nodes is reached

In [ ]:
api = tweepy.API(auth)
user_info = api.get_user('Vitaliy_Klychko')  
user_info._json['location']

In [ ]:
#use key words ['Kiev','Kyiv',]
acc_ntw = get_account_network(account_name, rel_type='followers', max_num =200, key_words=None)

In [70]:
cit_obl = ['Біла Церква', 'Бровари', 'Бориспіль', 'Фастів', 'Ірпінь',
        'Васильків', 'Боярка', 'Вишневе', 'Обухів',
        'Переяслав-Хмельницький', 'Буча', 'Славутич', 'Яготин', 'Вишгород',
        'Сквира', 'Березань', 'Богуслав', 'Тетіїв', 'Українка', 'Кагарлик',
        'Тараща', 'Миронівка', 'Узин', 'Ржищів', 'Чорнобиль', "Прип'ять"]

In [71]:
acc_ntw = get_account_network('huyova_bc', rel_type='followers', max_num =200, key_words=None)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:06


In [72]:
list_data = [(acc._json['followers_count'], 
  acc._json['statuses_count'], 
  acc._json['screen_name'], 
  acc._json['location'], 
  acc._json['lang']) for acc in acc_ntw]

df = pd.DataFrame(list_data, columns = ['followers_count','statuses_count','screen_name', 'location', 'lang'])

In [79]:
df_sorted = df.sort_values(by='followers_count', ascending=False)
df_sorted['location'].values

array(['Worldwide', 'Москва, Россия', 'Kyrzbekistan (Кырзбекистан)',
       'Ukraine ', 'Киев', 'Краматорск, Украина', 'Львів', 'Украина', '',
       'Kyiv', '', 'Україна', '', 'Україна', 'Zhytomyr', 'Украина',
       'SPACE', '', '', 'Санкт-Петербург, Россия', 'Украина', '', 'Минск',
       'Царствие Небесное', 'Украина -Донецк! ', 'Житомир', '',
       'Днепропетровск', 'МОСКВА ГРОБ', 'Aфины', '', '', '',
       'Ukraine, Dnepr', '', 'Івано-Франківськ0954573994', '',
       'Львів, Україна ', '', '', '', 'Ukraina', 'г. Николаев', 'Украина',
       '', '', '', '', 'Украина', '', 'Miami, FL', 'Detroit, MI',
       'United States', 'Україна', '', '', 'Украина', 'EU, Latvia', 'Киев',
       '', 'Oleshky, Ukraine', 'Украина', 'Украина, Николаев',
       'Динамо Київ', '', 'Россия', 'Dnipro', 'Россия', 'Харків', '', '',
       '', '', '', '', 'Старобільськ', 'борисполь',
       'Strawberry Fields, Ruthenia', '', 'Львів, Україна | Lviv, Ukraine',
       'Украина', 'Украина', 'Украина', '', 

In [47]:
acc_ntw[0]._json

{'blocked_by': False,
 'blocking': False,
 'contributors_enabled': False,
 'created_at': 'Sat Jul 14 14:47:34 +0000 2012',
 'default_profile': False,
 'default_profile_image': False,
 'description': '',
 'entities': {'description': {'urls': []}},
 'favourites_count': 40,
 'follow_request_sent': False,
 'followers_count': 1,
 'following': False,
 'friends_count': 24,
 'geo_enabled': True,
 'has_extended_profile': False,
 'id': 635511528,
 'id_str': '635511528',
 'is_translation_enabled': False,
 'is_translator': False,
 'lang': 'uk',
 'listed_count': 0,
 'live_following': False,
 'location': 'Чернігів',
 'muting': False,
 'name': 'ЇZHAK💙💛',
 'notifications': False,
 'profile_background_color': '000000',
 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme14/bg.gif',
 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme14/bg.gif',
 'profile_background_tile': False,
 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/635511528/14

### CATALAN COMARQUES

In [25]:
import pandas as pd

url = "https://en.wikipedia.org/wiki/Municipalities_of_Catalonia"
tabs = pd.read_html(url)[0]

tabs.columns = tabs.iloc[0]

tabs = tabs.iloc[1:]

In [80]:
tabs.keys()

Index(['Municipality', 'Comarca', 'Province', 'Population (2014)[3]',
       'Area (km2)[3]', 'Density', 'No. of EMDs[4]'],
      dtype='object', name=0)

In [35]:
tabs[tabs['Comarca'] == 'Osona']['Municipality'].values

array(['Alpens', 'Balenyà', 'El Brull', 'Calldetenes', 'Centelles',
       'Espinelves', "L'Esquirol", 'Folgueroles', 'Gurb', 'Lluçà', 'Malla',
       'Manlleu', 'Les Masies de Roda', 'Les Masies de Voltregà',
       'Montesquiu', 'Muntanyola', 'Olost', 'Orís', 'Oristà', 'Perafita',
       'Prats de Lluçanès', 'Roda de Ter', 'Rupit i Pruit',
       'Sant Agustí de Lluçanès', 'Sant Bartomeu del Grau',
       'Sant Boi de Lluçanès', 'Sant Hipòlit de Voltregà',
       'Sant Julià de Vilatorta', "Sant Martí d'Albars",
       'Sant Martí de Centelles', 'Sant Pere de Torelló',
       'Sant Quirze de Besora', "Sant Sadurní d'Osormort",
       'Sant Vicenç de Torelló', 'Santa Cecília de Voltregà',
       'Santa Eugènia de Berga', 'Santa Eulàlia de Riuprimer',
       'Santa Maria de Besora', 'Seva', 'Sobremunt', 'Sora', 'Taradell',
       'Tavèrnoles', 'Tavertet', 'Tona', 'Torelló', 'Vic', 'Vidrà',
       'Viladrau', 'Vilanova de Sau'], dtype=object)

In [82]:
tabs_sorted_com = tabs.sort_values(by='Comarca').set_index(['Comarca'])

In [85]:
tabs_sorted_com.ix['Osona']['Municipality'].values

array(['Muntanyola', 'Santa Eugènia de Berga', "Sant Sadurní d'Osormort",
       'Vidrà', 'Tona', 'Torelló', 'Rupit i Pruit', 'Roda de Ter',
       'Santa Cecília de Voltregà', 'Sant Vicenç de Torelló',
       'Santa Eulàlia de Riuprimer', 'Calldetenes', 'Prats de Lluçanès',
       'Tavertet', 'Montesquiu', 'Sant Julià de Vilatorta', 'Orís', 'Vic',
       'Olost', 'Sant Boi de Lluçanès', 'Perafita', "L'Esquirol",
       'Viladrau', 'Taradell', 'Les Masies de Roda', 'Sora',
       'Les Masies de Voltregà', 'Gurb', 'Seva', 'Sobremunt',
       'Folgueroles', 'Santa Maria de Besora', 'Oristà', 'Vilanova de Sau',
       'El Brull', 'Espinelves', "Sant Martí d'Albars",
       'Sant Bartomeu del Grau', 'Tavèrnoles', 'Manlleu', 'Malla',
       'Sant Pere de Torelló', 'Sant Quirze de Besora',
       'Sant Agustí de Lluçanès', 'Alpens', 'Sant Martí de Centelles',
       'Sant Hipòlit de Voltregà', 'Centelles', 'Balenyà', 'Lluçà'], dtype=object)